# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d065f0389f-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("FIFA World Cup events explorative search") 

Consider the following exploratory information need:

> Explore the participants and winners of different editions of the FIFA World Cup

## Useful URIs for the current workflow


The following are given:

| IRI           | Description    | Role      |
| -----------   | -----------    |-----------|
| `wdt:P1647`   | subproperty    | predicate |
| `wdt:P31`     | instance of    | predicate |
| `wdt:P106`    | profession     | predicate | 
| `wdt:P279`    | subclass       | predicate |
| `wdt:P27`     | nationality    | predicate |
| `wdt:P1532`   | country of sport    | predicate |
| `wd:Q29`      | Spain           | node |
| `wd:Q42267`   | Spain national football team | node |
| `wd:Q19317`   | FIFA World Cup  | node |
| `wd:Q176883`  | 2010 FIFA World cup  | node |
| `wd:Q208401`  | 2010 FIFA World Cup Final | node      |
| `wd:Q6979593`  |national football team | node      |


Also consider

```
?a wdt:P31 wd:Q6979593
```

is the BGP to retrieve all the  **national football teams**


## Workload Goals

1. Identify the BGP for connecting national football teams with the participations in the finals or in the world cups

2. Identify the BGP that connects a country with a national footbal team or as a location to a world cup

3. In which countries took place a FIFA world cup or world cup final? With how many participants?

4. Compare number of participants and winners of different world cups
 
   4.1 How many times each country participated in a world cup?
   
   4.2 Which country participated in the most finals? Which country won the most finals?
   
   4.3 Who participated in the most world cups among Italy, France, Spain, and Germany?


#### Example of query

In [2]:
queryString = """
SELECT COUNT(*) WHERE { 

    ?a wdt:P31 wd:Q6979593
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1220')]


1

### My Workflow

#### ***Task 1 :*** Identify the BGP for connecting national football teams with the participations in the finals or in the world cups.

As a starting point, I check if ***2010 FIFA World cup (wd:Q176883)*** is connected to ***Spain National football team (wd:Q42267)***.

In [3]:
queryString = """
SELECT ?p ?pName WHERE { 

    # Any property that connect 2010 FIFA World cup (wd:Q176883) and Spain National football team (wd:Q42267) 
    wd:Q176883 ?p wd:Q42267 .
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')]


2

They are connected by the property ***winner (wdt:P1346)*** and ***participating team (wdt:P1923)***

Since there is the property ***winner (wdt:P1346)*** there should be also a property between ***2010 FIFA World Cup Final (wd:Q208401)*** and ***Spain National football team (wd:Q42267)***. 

In [4]:
queryString = """
SELECT ?p ?pName WHERE { 

    # Any property that connect 2010 FIFA World Cup Final (wd:Q208401) and Spain National football team (wd:Q42267)
    wd:Q208401 ?p wd:Q42267 .
    
    # this returns the labels
    ?p <http://schema.org/name> ?pName .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')]


2

Indeed, we can see that the properties ***participating team (wdt:P1923)*** and ***winner (wdt:P1346)*** are used. 

The result is quite expected, because to win a competition you obviously need to partecipate to the final and win it.

Now, I want to retrieve other teams that partecipate in the FIFA World Cup. So, I show ***national football team (wd:wd:Q6979593)*** who partecipated in the ***2010 FIFA World Cup (wd:Q176883)***.

In [5]:
queryString = """
SELECT ?team ?teamName WHERE { 
 
    # Retrieve national football teams
    ?team wdt:P31 wd:Q6979593 .
    
    # Retrieve national football teams who partecipate in 2010 FIFA World Cup       
    wd:Q176883 wdt:P1923 ?team.  
            
    # This returns the labels
    ?team <http://schema.org/name> ?teamName .   
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q676899'), ('teamName', 'Italy national association football team')]
[('team', 'http://www.wikidata.org/entity/Q268208'), ('teamName', 'Australia national association football team')]
[('team', 'http://www.wikidata.org/entity/Q134916'), ('teamName', 'Uruguay national football team')]
[('team', 'http://www.wikidata.org/entity/Q170327'), ('teamName', 'South Africa national football team')]
[('team', 'http://www.wikidata.org/entity/Q14132'), ('teamName', 'North Korea national football team')]
[('team', 'http://www.wikidata.org/entity/Q174512'), ('teamName', 'Slovakia national association football team')]
[('team', 'http://www.wikidata.org/entity/Q175145'), ('teamName', 'Ivory Coast national football team')]
[('team', 'http://www.wikidata.org/entity/Q180187'), ('teamName', 'Honduras national football team')]
[('team', 'http://www.wikidata.org/entity/Q172221'), ('teamName', 'Slovenia national football team')]
[('team', 'http://www.wikidata.o

10

Ok, it works. But what if I want to work on different editions of the FIFA World Cup ? 

I need to understand how a specific edition (such as ***2010 FIFA World Cup (wd:Q176883)*** ) is connected to the general tournament ***FIFA World Cup (wd:Q19317)***.

In [6]:
queryString = """
SELECT ?p ?pName WHERE { 

    # Any property that connect 2010 FIFA World cup (wd:Q176883) to FIFA World Cup (wd:Q19317)
    wd:Q176883 ?p wd:Q19317 .
    
    # this returns the labels
    ?p <http://schema.org/name> ?pName .  
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pName', 'sports season of league or competition')]


1

Ok, so the property ***sports season of league or competition (wdt:P3450)*** is used to connnect a specific edition of the FIFA World Cup to the general tournament.

I also want to check how an edition of the FIFA World Cup is connected to its final. I use the 2010 edition, since I have both ***2010 FIFA World Cup (wd:Q176883)*** and ***2010 FIFA World Cup Final (wd:Q208401)***.

In [7]:
queryString = """
SELECT ?p ?pName WHERE { 

    # Any property that connect 2010 FIFA World cup (wd:Q176883) to 2010 FIFA World Cup Final (wd:Q208401)
    wd:Q176883 ?p wd:Q208401 .
    
    # this returns the labels
    ?p <http://schema.org/name> ?pName .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('pName', 'final event')]


1

The property ***final event (wdt:P3967)*** is used to connnect a specific edition of the FIFA World Cup to the final match of that specific edition.

If I want to retrieve national football teams that partecipated to a FIFA World Cup, I can rely on the properties ***sports season of league or competition (wdt:P3450)*** and ***participating team (wdt:P1923)*** .

In [8]:
queryString = """
SELECT DISTINCT ?teamName WHERE { 
       
    # Retrieve national football teams
    ?team wdt:P31 wd:Q6979593 .
            
    # Retrieve football teams who partecipate in a FIFA World Cup Edition     
    ?worldCupEdition wdt:P3450 wd:Q19317 ;
                     wdt:P1923 ?team     .
    
    # this returns the labels
    ?team <http://schema.org/name> ?teamName .  
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('teamName', 'Poland national association football team')]
[('teamName', 'Romania national association football team')]
[('teamName', 'Italy national association football team')]
[('teamName', 'Austria national association football team')]
[('teamName', 'Colombia national football team')]
[('teamName', 'Australia national association football team')]
[('teamName', 'Uruguay national football team')]
[('teamName', 'Bosnia and Herzegovina national football team')]
[('teamName', 'South Africa national football team')]
[('teamName', 'Iraq national football team')]
[('teamName', 'Israel national football team')]
[('teamName', 'North Korea national football team')]
[('teamName', 'Ukraine national association football team')]
[('teamName', 'Slovakia national association football team')]
[('teamName', 'Ivory Coast national football team')]
[('teamName', 'Honduras national football team')]
[('teamName', 'Wales national association football team')]
[('teamName', 'DR Congo national footba

20

Instead, if I want to retrieve only national football teams that partecipated to a final of the FIFA World Cup, I have to add the property ***final event (wdt:P3967)*** in order to retrieve the final match of a specific edition of ***FIFA World Cup (wd:Q19317)***.

In [9]:
queryString = """
SELECT DISTINCT ?teamName WHERE { 

    # Retrieve national football teams
    ?team wdt:P31 wd:Q6979593 .
            
    # Retrieve finals of FIFA World Cup editions  
    ?worldCupEdition    wdt:P3450 wd:Q19317 ;
                        wdt:P3967 ?final .
    
    # Retrieve national football teams partecipating to the final match
    ?final wdt:P1923 ?team . 
    
    # This returns the labels
    ?team <http://schema.org/name> ?teamName .   
} 
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('teamName', 'Italy national association football team')]
[('teamName', 'Uruguay national football team')]
[('teamName', 'Netherlands national association football team')]
[('teamName', 'France national association football team')]
[('teamName', 'Spain national association football team')]
[('teamName', 'Argentina national football team')]
[('teamName', 'Brazil national football team')]
[('teamName', 'Germany national association football team')]
[('teamName', 'Hungary national association football team')]
[('teamName', 'Czechoslovakia national association football team')]
[('teamName', 'Sweden national association football team')]


11

#### END TASK 1: 
I found both the requested BGPs:

The first one to retrieve national football teams that partecipated to a FIFA World Cup: 

```
?team               wdt:P31      wd:Q6979593 . 
?worldCupEdition    wdt:P3450    wd:Q19317   ; 
                    wdt:P1923    ?team       .
```

The second one to retrieve national football teams that partecipated to a final of the FIFA World Cup:

```
?team             wdt:P31     wd:Q6979593 .
?worldCupEdition  wdt:P3450   wd:Q19317   ; 
                  wdt:P3967   ?final      .   
?final            wdt:P1923   ?team       .
```

### ***Task 2 :*** Identify the BGP that connects a country with a national footbal team or as a location to a world cup

First, I show all the properties between ***Spain national football team (wd:Q42267)*** and ***Spain (wd:Q29)*** .

In [10]:
queryString = """
SELECT ?p ?pName WHERE { 

    # Any property that connect Spain National football team (wd:Q42267) and Spain (wd:Q29)
    wd:Q42267 ?p wd:Q29.
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .   
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pName', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]


2

I can use ***country for sport (wdt:P1532)*** that is also suggested as a useful URI.

I show some examples of couples [Football Team, Country].

In [11]:
queryString = """
SELECT DISTINCT ?teamName ?countryName WHERE { 

    # Search for instances of national football team and connect them to their country
    ?team   wdt:P31 wd:Q6979593 ;
            wdt:P1532 ?country .
    
    # This returns the labels
    ?team <http://schema.org/name> ?teamName .
    ?country <http://schema.org/name> ?countryName .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('teamName', 'South Vietnam national football team'), ('countryName', 'South Vietnam')]
[('teamName', 'South Vietnam national under-20 football team'), ('countryName', 'South Vietnam')]
[('teamName', 'Northern Mariana Islands national under-17 football team'), ('countryName', 'Northern Mariana Islands')]
[('teamName', 'Northern Mariana Islands national under-20 football team'), ('countryName', 'Northern Mariana Islands')]
[('teamName', 'Saaremaa official association football team'), ('countryName', 'Saaremaa')]
[('teamName', 'Frøya official association football team'), ('countryName', 'Frøya')]
[('teamName', 'Greenland national football team'), ('countryName', 'Greenland')]
[('teamName', 'Rhodes association football team'), ('countryName', 'Rhodes')]
[('teamName', 'Dominican Republic national football team'), ('countryName', 'Dominican Republic')]
[('teamName', 'Dominican Republic national under-20 football team'), ('countryName', 'Dominican Republic')]
[('teamName', 'Dominica

20

Now I want to discover how a country is connected as a location to a world cup.

I retrieve all the object properties connected to an edition of the ***FIFA World Cup (wd:Q19317)***.

In [12]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 
             
    # Search for properties connected to a FIFA World Cup Edition  
    ?worldCupEdition  wdt:P3450 wd:Q19317 ;
                      ?p        ?o .
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
    
    # Exclude data properties
    FILTER(!isLiteral(?o))
}
ORDER BY (?pName)
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('pName', 'final event')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P242'), ('pName', 'locator map image')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P822'), ('pName', 'mascot')]
[('p', 'http://www.wikidata.org/pr

28

The correct one may be one between ***country (wdt:P17)*** and ***location (wdt:P276)***. I try them using ***2010 FIFA World Cup (wd:Q176883)***.

In [13]:
queryString = """
SELECT ?country ?countryName WHERE { 

    # Connect the 2010 FIFA World cup (wd:Q176883) to something using property Contry (wdt:P17) 
    wd:Q176883 wdt:P17 ?country .
    
    # this returns the labels
    ?country <http://schema.org/name> ?countryName .
} 
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q258'), ('countryName', 'South Africa')]


1

Ok, so ***country (wdt:P17)*** is the correct one.

I retrieve some examples of couples [FIFA World Cup Edition, Country].

In [14]:
queryString = """
SELECT DISTINCT ?worldCupEditionName ?countryName WHERE { 
             
    # Search for countries that hosted a FIFA World Cup Edition 
    ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                      wdt:P17    ?country  .
    
    # this returns the labels
    ?worldCupEdition <http://schema.org/name> ?worldCupEditionName .
    ?country <http://schema.org/name> ?countryName .

}
ORDER BY ASC(?worldCupEditionName)
"""

print("Results")
run_query(queryString)

Results
[('worldCupEditionName', '1930 FIFA World Cup'), ('countryName', 'Uruguay')]
[('worldCupEditionName', '1934 FIFA World Cup'), ('countryName', 'Italy')]
[('worldCupEditionName', '1938 FIFA World Cup'), ('countryName', 'France')]
[('worldCupEditionName', '1950 FIFA World Cup'), ('countryName', 'Brazil')]
[('worldCupEditionName', '1954 FIFA World Cup'), ('countryName', 'Switzerland')]
[('worldCupEditionName', '1958 FIFA World Cup'), ('countryName', 'Sweden')]
[('worldCupEditionName', '1962 FIFA World Cup'), ('countryName', 'Chile')]
[('worldCupEditionName', '1966 FIFA World Cup'), ('countryName', 'United Kingdom')]
[('worldCupEditionName', '1970 FIFA World Cup'), ('countryName', 'Mexico')]
[('worldCupEditionName', '1974 FIFA World Cup'), ('countryName', 'Germany')]
[('worldCupEditionName', '1978 FIFA World Cup'), ('countryName', 'Argentina')]
[('worldCupEditionName', '1982 FIFA World Cup'), ('countryName', 'Mexico')]
[('worldCupEditionName', '1986 FIFA World Cup'), ('countryName',

26

#### END TASK 2:
I found both the requested BGPs:

The first one to retrieve countries related to a national football team: 
```
?team   wdt:P31    wd:Q6979593 ;
        wdt:P1532  ?country    .
```

The second one to retrieve national football teams that partecipated to a final of the FIFA World Cup:

```
?team             wdt:P31     wd:Q6979593 .
?worldCupEdition  wdt:P3450   wd:Q19317   ; 
                  wdt:P3967   ?final      .   
?final            wdt:P1923   ?team       .
```

### ***Task 3 :*** In which countries took place a FIFA world cup or world cup final? With how many participants?

To retrieve the number of participants I could rely on the property ***participating team (wdt:P1923)*** discovered before. 

However, it is not necessary to search both for ***FIFA World Cup (wd:Q19317)*** and FIFA World Cup Final since the results obtained by searching for the final matches are obviously included in the results obtained searching for ***FIFA World Cup (wd:Q19317)***.

In [15]:
queryString = """
SELECT ?worldCupEdition ?worldCupEditionName ?countryName ?numPartecipants WHERE { 
    
    # Connect a FIFA World Cup Edition to the hosting Country
    ?worldCupEdition wdt:P17 ?country .
    
    # this returns the labels
    ?worldCupEdition <http://schema.org/name> ?worldCupEditionName .
    ?country         <http://schema.org/name> ?countryName         .
    
    # Retrieving the number of partecipants for each FIFA World Cup Edition
    {   SELECT ?worldCupEdition COUNT(DISTINCT ?team) AS ?numPartecipants { 
            
            # Retrieve national football teams
            ?team wdt:P31 wd:Q6979593 .

            # Retrieve teams that partecipated to a FIFA World Cup Edition 
            ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                              wdt:P1923  ?team     .
        }
        GROUP BY ?worldCupEdition
    }  
} 
ORDER BY ASC(?worldCupEditionName)
"""

print("Results")
run_query(queryString)

Results
[('worldCupEdition', 'http://www.wikidata.org/entity/Q48432'), ('worldCupEditionName', '1930 FIFA World Cup'), ('countryName', 'Uruguay'), ('numPartecipants', '13')]
[('worldCupEdition', 'http://www.wikidata.org/entity/Q1477177'), ('worldCupEditionName', '1934 FIFA World Cup'), ('countryName', 'Italy'), ('numPartecipants', '16')]
[('worldCupEdition', 'http://www.wikidata.org/entity/Q131591'), ('worldCupEditionName', '1938 FIFA World Cup'), ('countryName', 'France'), ('numPartecipants', '15')]
[('worldCupEdition', 'http://www.wikidata.org/entity/Q132515'), ('worldCupEditionName', '1950 FIFA World Cup'), ('countryName', 'Brazil'), ('numPartecipants', '13')]
[('worldCupEdition', 'http://www.wikidata.org/entity/Q131075'), ('worldCupEditionName', '1954 FIFA World Cup'), ('countryName', 'Switzerland'), ('numPartecipants', '16')]
[('worldCupEdition', 'http://www.wikidata.org/entity/Q132533'), ('worldCupEditionName', '1958 FIFA World Cup'), ('countryName', 'Sweden'), ('numPartecipants'

21

#### END TASK 3:
With this query, I was able to associate every ***FIFA World Cup (wd:Q19317)*** Edition with its hosting country. 

Moreover, I also showed the number of partecipants of each edition ( which has grown up to 32 partecipants in the last editions).

(Just for curiosity: some editions have a strange number of partecipants because some teams retired from the final phase of the tournament due to financial issues or other problems)

### ***Task 4 :*** Compare number of participants and winners of different world cups.
I can rely on the properties: ***winner (wdt:P1346)*** and ***partecipating team (wdt:P1923)***.

In [16]:
queryString = """
SELECT ?wName ?numPartecipants ?winnerTeamName WHERE { 
    
    # Connect a FIFA World Cup Edition to the winner team
    ?worldCupEdition wdt:P1346 ?winnerTeam  .
    
        # This returns the labels
    ?worldCupEdition <http://schema.org/name> ?wName          .
    ?winnerTeam      <http://schema.org/name> ?winnerTeamName .
    
    # Retrieving the number of partecipants for each edition of the FIFA World Cup
    {   SELECT ?worldCupEdition COUNT(DISTINCT ?team) AS ?numPartecipants { 
            
            # Retrieve of national football team
            ?team wdt:P31 wd:Q6979593 .

            # Retrieve teams that partecipated to a FIFA World Cup Edition 
            ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                              wdt:P1923  ?team     .
        }
        GROUP BY ?worldCupEdition
    }
      
} 
ORDER BY ASC(?wName)
"""

print("Results")
run_query(queryString)

Results
[('wName', '1930 FIFA World Cup'), ('numPartecipants', '13'), ('winnerTeamName', 'Uruguay national football team')]
[('wName', '1934 FIFA World Cup'), ('numPartecipants', '16'), ('winnerTeamName', 'Italy national association football team')]
[('wName', '1938 FIFA World Cup'), ('numPartecipants', '15'), ('winnerTeamName', 'Italy national association football team')]
[('wName', '1950 FIFA World Cup'), ('numPartecipants', '13'), ('winnerTeamName', 'Uruguay national football team')]
[('wName', '1954 FIFA World Cup'), ('numPartecipants', '16'), ('winnerTeamName', 'Germany national association football team')]
[('wName', '1958 FIFA World Cup'), ('numPartecipants', '16'), ('winnerTeamName', 'Brazil national football team')]
[('wName', '1962 FIFA World Cup'), ('numPartecipants', '16'), ('winnerTeamName', 'Brazil national football team')]
[('wName', '1966 FIFA World Cup'), ('numPartecipants', '16'), ('winnerTeamName', 'England national association football team')]
[('wName', '1970 FIFA 

20

#### END TASK 4:
I correctly retrieved the winner of each edition of the ***FIFA World Cup (wd:Q19317)***.

### ***Task 4.1 :*** How many times each country participated in a world cup?

I can rely on the property ***partecipating team (wdt:P1923)*** to answer this question.

In [17]:
queryString = """
SELECT ?country ?countryName ?numEditions WHERE { 
    
    # Connect a national football team to its country
    ?team   wdt:P1532  ?country .
    
    # This returns the labels
    ?country <http://schema.org/name> ?countryName .
    
    # Retrieving the number of editions for each national football team
    {   SELECT ?team COUNT(DISTINCT ?worldCupEdition) AS ?numEditions { 
            
            # Retrieve national football teams
            ?team wdt:P31 wd:Q6979593 .

            # Retrieve teams that partecipated to a FIFA World Cup Edition 
            ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                              wdt:P1923  ?team .
        }
        GROUP BY ?team
    }
}
ORDER BY DESC(?numEditions)
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('numEditions', '20')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numEditions', '18')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numEditions', '18')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('numEditions', '16')]
[('country', 'http://www.wikidata.org/entity/Q96'), ('countryName', 'Mexico'), ('numEditions', '15')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numEditions', '14')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England'), ('numEditions', '14')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numEditions', '14')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('numEditions', '12')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('countryName'

80

#### END TASK 4.1:
I discovered that ***Brasil (wd:Q155)*** partecipated 20 different times to the ***FIFA World Cup (wd:Q19317)***.

It is followed by ***Italy (wd:Q38)*** and ***Germany (wd:Q183)*** with 18 times.

### ***Task 4.2 :*** Which country participated in the most finals? Which country won the most finals?

I can rely again on the properties ***winner (wdt:P1346)*** and ***partecipating team (wdt:P1923)***.

I have also to use ***final event (wdt:P3967)*** in order to retrieve the final match of a specific edition of ***FIFA World Cup (wd:Q19317)***.

In [18]:
queryString = """
SELECT ?country ?countryName COUNT(DISTINCT ?final) AS ?numFinals WHERE { 
    
    # Retrieve national football teams and their country
    ?team   wdt:P1532  ?country ;
            wdt:P31 wd:Q6979593 .

    # Retrieve finals of a FIFA World Cup Edition 
    ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                      wdt:P3967  ?final    .
    
    # Retrieve national football teams partecipating to the finals
    ?final wdt:P1923 ?team . 
     
    # This returns the labels
    ?country <http://schema.org/name> ?countryName .
}
GROUP BY ?country ?countryName
ORDER BY DESC(?numFinals)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numFinals', '6')]


1

For the second query, I simply substitute the property ***partecipating teams (wdt:P1923)*** with the property ***winner (wdt:P1346)***.

In [19]:
queryString = """
SELECT ?country ?countryName COUNT(DISTINCT ?final) AS ?numFinals WHERE { 
    
    # Retrieve national football teams and their country
    ?team   wdt:P1532  ?country    ;
            wdt:P31    wd:Q6979593 .

    # Retrieve finals of a FIFA World Cup Edition 
    ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                      wdt:P3967  ?final    .
    
    # Retrieve national football teams that won the finals
    ?final wdt:P1346 ?team . 
     
    # This returns the labels
    ?country <http://schema.org/name> ?countryName .
}
GROUP BY ?country ?countryName
ORDER BY DESC(?numFinals)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numFinals', '3')]


1

### ***Task 4.3 :***  Who participated in the most world cups among Italy, France, Spain, and Germany?

I have the URI code only for ***Spain National Football team (wd:Q42267)***. 

To retrieve the other URIs, I show all the national football teams that partecipated to at least one ***FIFA World Cup (wd:Q19317)*** Edition.

In [20]:
queryString = """
SELECT DISTINCT ?cName ?team ?tName WHERE { 

    # Search for instances of national football team connected to their country
    ?team   wdt:P31 wd:Q6979593 ;
            wdt:P1532 ?country .
    
    # Search for teams that partecipated to a FIFA World Cup Edition 
            ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                              wdt:P1923  ?team .
                              
    # This returns the labels
    ?country <http://schema.org/name> ?cName .
    ?team <http://schema.org/name> ?tName .
    
}
"""

print("Results")
run_query(queryString)

Results
[('cName', 'Ivory Coast'), ('team', 'http://www.wikidata.org/entity/Q175145'), ('tName', 'Ivory Coast national football team')]
[('cName', 'Cameroon'), ('team', 'http://www.wikidata.org/entity/Q175309'), ('tName', 'Cameroon national football team')]
[('cName', 'Morocco'), ('team', 'http://www.wikidata.org/entity/Q207337'), ('tName', 'Morocco national football team')]
[('cName', 'Nigeria'), ('team', 'http://www.wikidata.org/entity/Q181930'), ('tName', 'Nigeria national football team')]
[('cName', 'Senegal'), ('team', 'http://www.wikidata.org/entity/Q207441'), ('tName', 'Senegal national association football team')]
[('cName', 'Ghana'), ('team', 'http://www.wikidata.org/entity/Q172014'), ('tName', 'Ghana national football team')]
[('cName', 'France'), ('team', 'http://www.wikidata.org/entity/Q47774'), ('tName', 'France national association football team')]
[('cName', "People's Republic of China"), ('team', 'http://www.wikidata.org/entity/Q130582'), ('tName', 'China national footb

80

Ok, now I have:
* ***Italy national association football team (wd:Q676899)***
* ***Germany national association football team (wd:Q43310)***
* ***France national association football team (wd:Q47774)***

Now, I can use the same query used for Task 4.1, but filtering the results using the just retrieved information.

In [21]:
queryString = """
SELECT ?countryName ?numEditions WHERE { 
    
    # Connect a national football team to its country
    ?team wdt:P1532 ?country .
    
    # This returns the labels
    ?country <http://schema.org/name> ?countryName .
    
    # Retrieving the number of editions for each national football team
    {   SELECT  ?team COUNT(DISTINCT ?worldCupEdition) AS ?numEditions { 
            
            # Retrieve national football team
            ?team wdt:P31 wd:Q6979593 .

            # Retrieve teams that partecipated to a FIFA World Cup Edition 
            ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                              wdt:P1923  ?team .
                              
            # I want only Spain, Italy, Germany and France             
            FILTER(?team = wd:Q42267 || ?team = wd:Q676899 || ?team = wd:Q43310 || ?team = wd:Q47774 )                  
        }
        GROUP BY ?team
    }
}
ORDER BY DESC(?numEditions)
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Germany'), ('numEditions', '18')]
[('countryName', 'Italy'), ('numEditions', '18')]
[('countryName', 'France'), ('numEditions', '14')]
[('countryName', 'Spain'), ('numEditions', '14')]


4

#### END TASK 4.3:
Both ***Italy (wd:Q38)*** and ***Germany (wd:Q183)*** partecipated 18 times to a ***FIFA World Cup (wd:Q19317)*** Edition.

### Extra analytics query

#### For each national football team that won FIFA World Cup at least once, show the ratio between ***# FIFA World Cup won*** and ***# FIFA World Cup partecipation***.

In [22]:
queryString = """
SELECT ?teamName (100*?numWin/?numEditions) as ?winRate  WHERE { 
    
    # This returns the labels
    ?team <http://schema.org/name> ?teamName .
    
    # Retrieving the number of participations or each national football team
    {   SELECT ?team COUNT(DISTINCT ?worldCupEdition) AS ?numEditions { 
            
            # Retrieve national football teams
            ?team wdt:P31 wd:Q6979593 .

            # Retrieve teams that partecipated to a FIFA World Cup Edition 
            ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                              wdt:P1923  ?team .
        }
        GROUP BY ?team
    }
    
    # Retrieve number of win for each national football team
    {    SELECT ?team COUNT(DISTINCT ?worldCupEdition) AS ?numWin WHERE { 
    
            # Retrieve national football teams
            ?team  wdt:P31 wd:Q6979593 .

            # Retrive teams that won a FIFA World Cup Edition 
            ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                              wdt:P1346  ?team .
        }
        GROUP BY ?team 
    }
} 
ORDER BY DESC(?winRate)
"""

print("Results")
run_query(queryString)

Results
[('teamName', 'Brazil national football team'), ('winRate', '25')]
[('teamName', 'Italy national association football team'), ('winRate', '22')]
[('teamName', 'Germany national association football team'), ('winRate', '22')]
[('teamName', 'Uruguay national football team'), ('winRate', '16')]
[('teamName', 'France national association football team'), ('winRate', '14')]
[('teamName', 'Argentina national football team'), ('winRate', '12')]
[('teamName', 'England national association football team'), ('winRate', '7')]
[('teamName', 'Spain national association football team'), ('winRate', '7')]


8

#### Show the total number of partecipations grouped by Continent.
First, I need to understand how a country and a continent are connected. So I show all the properties connected to ***Spain (wd:Q29)***.

In [23]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Any property that connect Spain (wd:Q29) with something.
    wd:Q29 ?p ?o.
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .   
    
    # Exclude data properties
    FILTER(!isLiteral(?o))
    
}
ORDER BY ?pName
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5125'), ('pName', 'Wikimedia outline')]
[('p', 'http://www.wikidata.org/prop/direct/P85'), ('pName', 'anthem')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pName', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('pName', 'capital')]
[('p', 'http://www.wikidata.org/prop/direct/P1740'), ('pName', 'category for films shot at this location')]
[('p', 'http://www.wikidata.org/prop/direct/P7867'), ('pName', 'category for maps')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('pName', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P1465'), ('pName', 'category for people who died here')]
[('p', 'http://www.wikidata.org/prop/direct/P1792'), ('pName', 'category of associated people')]
[('p', 'http://www.wikidata.org/prop/direct/P1791'), ('pName', 'category of people buried here')]
[('p', 'http://www.wikidata.org/prop/direct/P237'), ('pName', 'coat of arms')]
[('p',

30

The property ***continent (wdt:P30)*** is used to connect a Country to the corresponding Continent.

In [24]:
queryString = """
SELECT ?continentName SUM(?numEditions) AS ?totPartecipationsByContinent  WHERE { 
  
    # Connect a national football team to its country
    ?team wdt:P1532 ?country .
    
    # Connect a country to the continent
    ?country wdt:P30 ?continent .
  
    # This returns the labels
    ?continent <http://schema.org/name> ?continentName .
    
    # Retrieving the number of partecipations for each national football team
    {   SELECT ?team COUNT(DISTINCT ?worldCupEdition) AS ?numEditions { 
            
            # Retrieve national football teams
            ?team wdt:P31 wd:Q6979593 .

            # Retrieve teams that partecipated to a FIFA World Cup Edition 
            ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                              wdt:P1923  ?team .
        }
        GROUP BY ?team
    }
}
GROUP BY ?continentName
ORDER BY DESC(?totPartecipationsByContinent)
"""

print("Results")
run_query(queryString)

Results
[('continentName', 'Europe'), ('totPartecipationsByContinent', '223')]
[('continentName', 'South America'), ('totPartecipationsByContinent', '80')]
[('continentName', 'Asia'), ('totPartecipationsByContinent', '44')]
[('continentName', 'North America'), ('totPartecipationsByContinent', '43')]
[('continentName', 'Africa'), ('totPartecipationsByContinent', '39')]
[('continentName', 'Insular Oceania'), ('totPartecipationsByContinent', '16')]
[('continentName', 'Central America'), ('totPartecipationsByContinent', '10')]
[('continentName', 'Eurasia'), ('totPartecipationsByContinent', '2')]


8

There are some strange continents since Wikidata is not consistant when using ***continent (wd:Q5107)*** due to the different conventions to identify continents. 

#### Attendance at finals
I want to discover more statistics about the finals, so I show all the properties connected to a final of the ***FIFA World Cup (wd:Q19317)***.

In [25]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 
    
    # Retrieve finals of a FIFA World Cup Edition 
    ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                      wdt:P3967  ?final .
    
    # Everything that is connected with a final of the FIFA World Cup Edition
    ?final ?p ?o .
     
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7494'), ('pName', 'Soccerway match ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1363'), ('pName', 'points/goal scored by')]
[('p', 'http://www.wikidata.org/prop/direct/P1652'), ('pName', 'referee')]
[('p', 'http://www.wikidata.org/prop/direct/P7492'), ('pName', 'EU-Football.info match ID')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different fro

36

For example, I can now use the property ***attendance (wdt:P1110)*** to show the number of attendants of the final events.

In [26]:
queryString = """
SELECT ?final ?finalName ?attendance WHERE { 
    
    # Retrieve finals of a FIFA World Cup Edition 
    ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                      wdt:P3967  ?final .
    
    # Retrieve the attendance value of the final
    ?final wdt:P1110 ?attendance .
    
    # this returns the labels
    ?final <http://schema.org/name> ?finalName .
}
ORDER BY DESC(?finalName)
"""

print("Results")
run_query(queryString)

Results
[('final', 'http://www.wikidata.org/entity/Q754483'), ('finalName', 'Uruguay v Brazil 1950'), ('attendance', '199854')]
[('final', 'http://www.wikidata.org/entity/Q31043671'), ('finalName', '2018 FIFA World Cup Final'), ('attendance', '78011')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('finalName', '2014 FIFA World Cup Final'), ('attendance', '74738')]
[('final', 'http://www.wikidata.org/entity/Q208401'), ('finalName', '2010 FIFA World Cup Final'), ('attendance', '84490')]
[('final', 'http://www.wikidata.org/entity/Q268567'), ('finalName', '2006 FIFA World Cup Final'), ('attendance', '69000')]
[('final', 'http://www.wikidata.org/entity/Q715021'), ('finalName', '2002 FIFA World Cup Final'), ('attendance', '69029')]
[('final', 'http://www.wikidata.org/entity/Q585295'), ('finalName', '1998 FIFA World Cup Final'), ('attendance', '80000')]
[('final', 'http://www.wikidata.org/entity/Q389104'), ('finalName', '1994 FIFA World Cup Final'), ('attendance', '94194')]
[('fina

21

#### Compute the averange attendance of the FIFA World Cup Finals.

In [27]:
queryString = """
SELECT AVG(?attendance) as ?avgAttendance WHERE { 
    
    # Retrieve finals of a FIFA World Cup Edition 
    ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                      wdt:P3967  ?final .
    
    # Retrieve the attendance value of the final
    ?final wdt:P1110 ?attendance .
    
    # this returns the labels
    ?final <http://schema.org/name> ?finalName .
}
"""

print("Results")
run_query(queryString)

Results
[('avgAttendance', '82276.19047619047619')]


1

#### Show the FIFA World Cup Final with the highest attendance.

In [28]:
queryString = """
SELECT ?final ?finalName ?attendance WHERE { 
    
    # Retrieve finals of a FIFA World Cup Edition 
    ?worldCupEdition  wdt:P3450  wd:Q19317 ;
                      wdt:P3967  ?final .
    
    # Retrieve the attendance value of the final
    ?final wdt:P1110 ?attendance .
    
    # this returns the labels
    ?final <http://schema.org/name> ?finalName .
}
ORDER BY DESC(?attendance)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('final', 'http://www.wikidata.org/entity/Q754483'), ('finalName', 'Uruguay v Brazil 1950'), ('attendance', '199854')]


1

## END